## Evaluation of Weekly Performance based on MASE & SMAPE

In [2]:
!pip install utilsforecast -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.1 MB/s eta 0:00:00


In [7]:
import pandas as pd
from utilsforecast.losses import smape, mase

In [8]:
train = pd.read_excel("VN1_train.xlsx")
actuals=pd.read_csv("Test_Weekly_VN1_Actuals_final.csv")
forecast=pd.read_csv("final_forecast_weekly_merged.csv")

In [9]:
train["ds"]=pd.to_datetime(train["ds"])
actuals["ds"]=pd.to_datetime(actuals["ds"])
forecast["ds"]=pd.to_datetime(forecast["ds"])

In [10]:
def evaluate(train, actuals, forecast, seasonality=52, id_col='unique_id', target_col='y'):
  fcst_and_actuals = pd.merge(actuals, forecast, on=['ds', 'unique_id'], how='left')
  models=[ "AutoARIMA", "AutoETS", "AutoTBATS", 'RandomForestRegressor','XGBRegressor', "LGBMRegressor", "TFT", "NHITS", "TimeGPT",'TimeGPT_finetuned', "TimesFM"]
  mase_val = mase(fcst_and_actuals, models=models, seasonality=seasonality, train_df=train, id_col='unique_id', target_col='y')
  mase_val = mase_val.mean(numeric_only=True)
  smape_val = smape(fcst_and_actuals, models=models,  id_col='unique_id', target_col='y')
  smape_val = smape_val.mean(numeric_only=True)
  results = pd.DataFrame({'mase': mase_val, 'smape': smape_val}, index=models)
  results.index.name = 'models'
  results = results.round(4)
  return results

In [11]:
weekly_results=evaluate(train, actuals, forecast, seasonality=52, id_col='unique_id', target_col='y')

In [13]:
weekly_results

,MASE,SMAPE
models,,
AutoARIMA,0.2912,0.1625
AutoETS,0.2582,0.1542
AutoTBATS,0.2981,0.1637
RandomForestRegressor,0.2796,0.1534
XGBRegressor,0.2702,0.1522
LGBMRegressor,0.2888,0.1551
TFT,0.2081,0.1482
NHITS,0.2082,0.1550
TimeGPT,0.2510,0.1586
